#Instalando e importando as bibliotecas

In [ ]:
# importa libraries necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import warnings
warnings.filterwarnings('ignore')


#Carregando os dados da planilha

In [ ]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download dos dados para esta máquina virtual
downloaded = drive.CreateFile({'id': '1ZOspzswkB75WFfSg23GrCMdqIOf0EXOH'})
downloaded.GetContentFile('data.csv')

In [ ]:
# Carregando os dados baixados no dataframe 
df = pd.read_csv('data.csv')

In [ ]:
#olhamos as colunas do df
df.columns

Index(['anomes', 'num_cpf_hash', 'vlr_credito', 'vlr_saldo', 'num_atend_atrs',
       'vlr_score', 'num_produtos', 'num_atend', 'qtd_oper', 'qtd_reclm',
       'qtd_restr', 'vlr_renda', 'cod_rating', 'ind_atrito', 'ind_engaj',
       'ind_novo_cli'],
      dtype='object')

#Criação de um data frame com um cliente por linha que em algum momento foram engajados com o banco, resumindo a sua trajetória durante os 12 mêses fornecidos pelo banco 


<h2>1) Precisamos pegar toda a jornada dos clientes que em algum momento ja foram engajados com o banco

<p>1º) Criamos um df que contenha apenas um cpf por linha de pessoas que possuem o indice engajado como 1, independente do anomes

In [ ]:
#df com pessoas que em algum momentos foram engajadas
df_engajado = df[(df.ind_engaj ==1)].drop_duplicates("num_cpf_hash")

<p>2º) Criamos um df que pegará todos os anomes daquelas pessoas que possuem indice engajado como 1, independente se naquele anomes o indice engajado estava 1 ou 0.

In [ ]:
# criação de um df, que não é um ponteiro para outro df, que possui os dados de todos os anomes dos clientes ja foram engajados em algum momento, eliminando as colunas "vlr_renda", "ind_novo_cli" e "anomes". Essas colunas foram removidas pois acreditamos que não agregam para nosso resultado de indice de engajamento. Também já substituimos os valores Nan por 0. 
clientes_engajados_jornada  = df[df["num_cpf_hash"].isin(df_engajado.num_cpf_hash.values)].fillna(0).drop(["vlr_renda", "ind_novo_cli", "anomes",], axis=1).copy()

,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,cod_rating,ind_atrito,ind_engaj
1,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,35943.74,5815.50,0.0,377.0,1.0,0.0,24.0,0.0,10.0,A,0.0,1.0
4,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,1238.93,1400.00,0.0,773.0,2.0,0.0,21.0,0.0,0.0,A,0.0,1.0
8,ffff4ef886c28af8029c6b7d504942e6c9ef13021e9a35...,81506.54,5243.45,0.0,560.0,1.0,0.0,14.0,0.0,0.0,A,0.0,0.0
9,ffff4bbb4074d7894174fd94c685edb3fd7fc9b7b1e3c1...,112918.62,7252.64,0.0,374.0,2.0,0.0,23.0,0.0,0.0,A,0.0,1.0
16,fffe7c6f3755240f56780edde1f91b70de574bf4e6e767...,9806.33,1752.18,0.0,611.0,2.0,0.0,5.0,0.0,0.0,A,0.0,1.0


In [ ]:
# deletamos algumas variáveis para liberar espaço na memória ram
del df
del df_engajado

In [ ]:
# verificamos se todas as colunas que desejamos está no df
clientes_engajados_jornada.columns

Index(['num_cpf_hash', 'vlr_credito', 'vlr_saldo', 'num_atend_atrs',
       'vlr_score', 'num_produtos', 'num_atend', 'qtd_oper', 'qtd_reclm',
       'qtd_restr', 'cod_rating', 'ind_atrito', 'ind_engaj'],
      dtype='object')

In [ ]:
# Checagem de quantas linhas temos no df que acabamos de criar
len(clientes_engajados_jornada)

3338100

<h2>2) Precisamos filtrar essas informações aplicando algumas "feature engineering" e resumir os clientes em uma única linha

<p>1º) Criar um label encoder para a coluna que temos string como valor

In [ ]:
# Checagem de quais valores temos na coluna "cod_rating"
clientes_engajados_jornada["cod_rating"].unique()

array(['A', 'C', 'B', 0, 'E', 'D', 'F', 'H', 'HH', 'G', 'AA'],
      dtype=object)

In [ ]:
 # aplica label encode em cod_rating já que o valor da coluna não é numérico,mas sim texto
clientes_engajados_jornada["cod_rating"] = clientes_engajados_jornada["cod_rating"].astype('category')

#poem os novos valores na coluna
clientes_engajados_jornada["cod_rating"] = clientes_engajados_jornada["cod_rating"].cat.codes
clientes_engajados_jornada["cod_rating"].unique()

array([ 1,  4,  3,  0,  6,  5,  7,  9, 10,  8,  2], dtype=int8)

<p>2º) Ajuste de incongruência nos dados de saldo e número de produtos

In [ ]:
#  filtramos clientes que tem saldo mas não tem produtos já que ter saldo é um produto. Substituimos 0 por 1(não realizamos a mudança pois ainda não confirmamos com o cliente se nossa hipótese está certa)

# clientes_engajados_jornada.isin(clientes_engajados_jornada[(clientes_engajados_jornada["num_produtos"] ==0.0) & (clientes_engajados_jornada["vlr_saldo"] !=0)])["num_produtos"].replace(to_replace=0.0,value=1,inplace=True)
#clientes_engajados_jornada[(clientes_engajados_jornada["num_produtos"] ==0.0) & (clientes_engajados_jornada["vlr_saldo"] !=0)]



<p>3º) Realização de soma, mediana, média e valor máximo para cada histórico dos clientes ao longo dos anomes disponíveis de cada

<p>Realizamos os seguintes agrupamentos para as seguintes colunas:
<p>-------------------------
<p>Soma:
<p>qtd_oper
<p>qtd_reclm
<p>num_atend_atrs
<p>num_atend
<p>qtd_restr
<p>-------------------------
<p>Mediana:
<p>vlr_credito
<p>vlr_saldo
<p>num_produtos
<p>ind_engaj
<p>cod_rating
<p>-------------------------
<p>Máximo:
<p>ind_atrito
<p>-------------------------
<h5>Média:
<p>vlr_score



In [ ]:
# Criação de uma função para realizar os agrupamentos que precisamos
def agrupar_dados(this_df):
    #Somando os valores
    this_df["qtd_oper"] = this_df.qtd_oper.sum()
    this_df["qtd_reclm"] = this_df.qtd_reclm.sum()
    this_df["num_atend_atrs"] = this_df.num_atend_atrs.sum()
    this_df["num_atend"] = this_df.num_atend.sum()
    this_df["qtd_restr"] = this_df.qtd_restr.sum()
    
    #Pegando a mediana dos valores
    this_df["vlr_credito"] = this_df.vlr_credito.median()
    this_df["vlr_saldo"] = this_df.vlr_saldo.median()
    this_df["num_produtos"] = this_df.num_produtos.median()
    this_df["cod_rating"] = this_df.cod_rating.median()
    this_df["ind_engaj"] = this_df.ind_engaj.median()

    #Pegando a média dos valores
    this_df["vlr_score"] = this_df.vlr_score.mean()

    #Pegando o máximo dos valores
    this_df["ind_atrito"] = this_df.ind_atrito.max()
   
   #Retornamos o df modificado
    return this_df


In [ ]:
# Aplicamos a função de criamos em um groupby para que as agregações sejam feitas para cada cpf (obs.: esse passo está demorando cerca de 13 minutos para ser concluido)
clientes_engajados_jornada = clientes_engajados_jornada.groupby("num_cpf_hash").apply(agrupar_dados)

,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,cod_rating,ind_atrito,ind_engaj
1,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,34690.20,5780.11,0.0,437.615385,2.0,0.0,193.0,0.0,64.0,1.0,0.0,1.0
4,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,4086.83,1400.27,0.0,749.076923,2.0,0.0,411.0,0.0,0.0,1.0,0.0,1.0
8,ffff4ef886c28af8029c6b7d504942e6c9ef13021e9a35...,78661.03,5220.94,0.0,486.076923,1.0,0.0,187.0,0.0,0.0,1.0,0.0,1.0
9,ffff4bbb4074d7894174fd94c685edb3fd7fc9b7b1e3c1...,116531.01,6663.52,0.0,359.307692,2.0,0.0,290.0,0.0,8.0,1.0,0.0,1.0
16,fffe7c6f3755240f56780edde1f91b70de574bf4e6e767...,9660.02,1752.27,0.0,553.153846,1.0,0.0,97.0,0.0,0.0,1.0,0.0,1.0


<p>4º) Substituição de valores errados após o resumo de cada cliente

In [ ]:
#Retiramos os clientes duplicados e resetamos index
clientes_engajados_jornada = clientes_engajados_jornada.drop_duplicates("num_cpf_hash")
clientes_engajados_jornada = clientes_engajados_jornada.reset_index(drop=True)

(257803, 13)

In [ ]:
# Encontramos valores de 0.5 para o indice engajado em detrimento da aplicação da mediana. Para solucionar esse problema, estamos substituindo para 1 uma vez que precisamos mais de valores engajados e esse usuário esteve 50% do tempo engajado.
clientes_engajados_jornada["ind_engaj"].replace(to_replace=0.5,value=1,inplace=True)

<p>5º) Checagem dos resultados

In [ ]:
#Verificamos qual a porcentagem de pessoas que no resultado final ficaram como engajados
len(clientes_engajados_jornada[(clientes_engajados_jornada.ind_engaj == 1)]) / len(clientes_engajados_jornada)

0.6411329581114261

In [ ]:
#verificamos se há apenas dois valores para "ind_engaj", ou 0 ou 1.
len(clientes_engajados_jornada[(clientes_engajados_jornada.ind_engaj == 1) | (clientes_engajados_jornada.ind_engaj == 0)]) / len(clientes_engajados_jornada)

1.0

In [ ]:
#Verificando quantos clientes obtivemos no final
len(clientes_engajados_jornada)

257803

In [ ]:
#Verificando os resultados nos primeiros 10 clientes apenas
clientes_engajados_jornada.head(10)

,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,cod_rating,ind_atrito,ind_engaj
0,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,34690.20,5780.11,0.0,437.615385,2.0,0.0,193.0,0.0,64.0,1.0,0.0,1.0
1,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,4086.83,1400.27,0.0,749.076923,2.0,0.0,411.0,0.0,0.0,1.0,0.0,1.0
2,ffff4ef886c28af8029c6b7d504942e6c9ef13021e9a35...,78661.03,5220.94,0.0,486.076923,1.0,0.0,187.0,0.0,0.0,1.0,0.0,1.0
3,ffff4bbb4074d7894174fd94c685edb3fd7fc9b7b1e3c1...,116531.01,6663.52,0.0,359.307692,2.0,0.0,290.0,0.0,8.0,1.0,0.0,1.0
4,fffe7c6f3755240f56780edde1f91b70de574bf4e6e767...,9660.02,1752.27,0.0,553.153846,1.0,0.0,97.0,0.0,0.0,1.0,0.0,1.0
5,fffe21c8e200dbb6dddf1f99b8f6910f82338c9e4c1b49...,41559.72,25113.60,0.0,427.076923,1.0,0.0,144.0,0.0,181.0,1.0,0.0,1.0
6,fffdfca27ab85ba0973fbb09469c7d4f0c8c2a8fc94852...,479213.00,33852.95,0.0,199.846154,3.0,0.0,428.0,0.0,140.0,1.0,0.0,1.0
7,fffded88432c844360bbf1323077c2691b744725d60fa1...,7582.37,1075.25,0.0,605.846154,4.0,0.0,207.0,0.0,0.0,1.0,0.0,1.0
8,fffdbe204fb2f4427dc75ce792aa6eafbfd1e32d4e865c...,30767.57,509.59,0.0,278.307692,2.0,1.0,166.0,0.0,22.0,1.0,0.0,1.0
9,fffd8b2d6282071aa093603ee377ba1727f26d6a4d6e92...,9061.27,10999.85,0.0,617.923077,2.0,0.0,242.0,0.0,0.0,1.0,0.0,1.0


<p>6º) Baixamos o df como csv no drive do grupo para não executarmos essa função que demora muito toda vez que precisarmos desse df

In [ ]:
# Importamos a biblioteca drive do google.colab
from google.colab import drive

# Estabelecemos os caminhos para o download poder ser executado
drive.mount(r'/content/drive')
path = r'/content/drive/My Drive/Projeto_Banco_Pan_Grupo_1/df_filtro_clientes_engaj.csv'

# Baixando nosso df no formato de csv em nosso drive
with open(path, 'w', encoding = 'utf-8-sig') as f:
  clientes_engajados_jornada.to_csv(f)

Mounted at /content/drive
